<a href="https://colab.research.google.com/github/martindevoto/machine-learning-notebooks-personal/blob/main/Intro_Haystack_pt_8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Preprocessing

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/deepset-ai/haystack/blob/master/tutorials/Tutorial8_Preprocessing.ipynb)

Haystack includes a suite of tools to extract text from different file types, normalize white space
and split text into smaller pieces to optimize retrieval.
These data preprocessing steps can have a big impact on the systems performance and effective handling of data is key to getting the most out of Haystack.

Ultimately, Haystack expects data to be provided as a list documents in the following dictionary format:
``` python
docs = [
    {
        'content': DOCUMENT_TEXT_HERE,
        'meta': {'name': DOCUMENT_NAME, ...}
    }, ...
]
```

This tutorial will show you all the tools that Haystack provides to help you cast your data into this format.

In [ ]:
# Install the latest release of Haystack in your own environment
#! pip install farm-haystack

# Install the latest master of Haystack
!pip install --upgrade pip
!pip install git+https://github.com/deepset-ai/haystack.git#egg=farm-haystack[colab,ocr]

!wget --no-check-certificate https://dl.xpdfreader.com/xpdf-tools-linux-4.03.tar.gz
!tar -xvf xpdf-tools-linux-4.03.tar.gz && sudo cp xpdf-tools-linux-4.03/bin64/pdftotext /usr/local/bin

     |████████████████████████████████| 2.1 MB 4.2 MB/s 
  Attempting uninstall: pip
    Found existing installation: pip 21.1.3
    Uninstalling pip-21.1.3:
      Successfully uninstalled pip-21.1.3
  Cloning https://github.com/deepset-ai/haystack.git to /tmp/pip-install-okyfk4nk/farm-haystack_6e18cba158fb476ca87145ab2bac34b9
  Running command git clone --filter=blob:none --quiet https://github.com/deepset-ai/haystack.git /tmp/pip-install-okyfk4nk/farm-haystack_6e18cba158fb476ca87145ab2bac34b9
  Resolved https://github.com/deepset-ai/haystack.git to commit aca52ea39c4923a9a1c03172c1d22da7f60e505f
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 359.7/359.7 KB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 KB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 64.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━

In [ ]:
# Here are the imports we need
from haystack.nodes import (TextConverter, PDFToTextConverter,
                            DocxToTextConverter, PreProcessor)
from haystack.utils import convert_files_to_dicts, fetch_archive_from_http

In [ ]:
# This fetches some sample files to work with

doc_dir = 'data/preprocessing_tutorial'
s3_url = "https://s3.eu-central-1.amazonaws.com/deepset.ai-farm-qa/datasets/documents/preprocessing_tutorial.zip"
fetch_archive_from_http(url=s3_url, output_dir=doc_dir)

INFO - haystack.utils.import_utils -  Fetching from https://s3.eu-central-1.amazonaws.com/deepset.ai-farm-qa/datasets/documents/preprocessing_tutorial.zip to `data/preprocessing_tutorial`


True

## Converters

Haystack's converter classes are designed to help you turn files on your computer into the documents
that can be processed by the Haystack pipeline.
There are file converters for txt, pdf, docx files as well as a converter that is powered by Apache Tika.
The parameter `valid_langugages` does not convert files to the target language, but checks if the conversion worked as expected.
For converting PDFs, try changing the encoding to UTF-8 if the conversion isn't great.

In [ ]:
# Here are some examples of how you would use file converters

converter = TextConverter(remove_numeric_tables=True, 
                          valid_languages=['en'])
doc_txt = converter.convert(file_path='data/preprocessing_tutorial/classics.txt',
                            meta=None)[0]

converter = PDFToTextConverter(remove_numeric_tables=True,
                               valid_languages=['en'])
doc_pdf = converter.convert(file_path='data/preprocessing_tutorial/bert.pdf',
                            meta=None)[0]

converter = DocxToTextConverter(remove_numeric_tables=False,
                                valid_languages=['en'])
doc_docx = converter.convert(file_path='data/preprocessing_tutorial/heavy_metal.docx',
                             meta=None)[0]

In [ ]:
# Haystack also has a convenience function that will automatically apply
# the right converter to each file in a directory

all_docs = convert_files_to_dicts(dir_path='data/preprocessing_tutorial')

INFO - haystack.utils.preprocessing -  Converting data/preprocessing_tutorial/heavy_metal.docx
INFO - haystack.utils.preprocessing -  Converting data/preprocessing_tutorial/classics.txt
INFO - haystack.utils.preprocessing -  Converting data/preprocessing_tutorial/bert.pdf


## PreProcessor

The PreProcessor class is designed to help you clean text and split text into sensible units.
File splitting can have a very significant impact on the system's performance and is absolutely mandatory for Dense Passage Retrieval models.
In general, we recommend you split the text from your files into small documents of around 100 words for dense retrieval methods
and no more than 10,000 words for sparse methods.
Have a look at the [Preprocessing](https://haystack.deepset.ai/docs/latest/preprocessingmd)
and [Optimization](https://haystack.deepset.ai/docs/latest/optimizationmd) pages on our website for more details.

In [ ]:
# This a default usage of the PreProcessor
# Here, it performs cleaning of consecutive whitespaces
# and splits a single large document into smaller documents
# Each document is up to 1000 words long and document breaks cannot fall in the
# middle of sentences
# Note how the single document passed into the document gets split into
# 5 smaller documents

preprocessor = PreProcessor(
    clean_empty_lines=True,
    clean_whitespace=True,
    clean_header_footer=False,
    split_by='word',
    split_length=100,
    split_respect_sentence_boundary=True,
)

docs_default = preprocessor.process(doc_txt)
print(f'n_docs_input: 1\nn_docs_output: {len(docs_default)}')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
n_docs_input: 1
n_docs_output: 51


## Cleaning

- `clean_empty_lines` will normalize 3 or more consecutive empty lines to be just a two empty lines
- `clean_whitespace` will remove any whitespace at the beginning or end of each line in the text
- `clean_header_footer` will remove any long header or footer texts that are repeated on each page

## Splitting
By default, the PreProcessor will respect sentence boundaries, meaning that documents will not start or end
midway through a sentence.
This will help reduce the possibility of answer phrases being split between two documents.
This feature can be turned off by setting `split_respect_sentence_boundary=False`.

In [ ]:
# Not respecting sentence boundary vs. respecting sentence boundary

preprocessor_nrsb = PreProcessor(split_respect_sentence_boundary=False)
docs_nrsb = preprocessor_nrsb.process(doc_txt)

print('Respecting sentence boundary')
end_text = docs_default[0]['content'][-50:]
print('End of document: "...' + end_text + '"')
print()
print('Not respecting sentence boundary')
end_text_nrsb = docs_nrsb[0]['content'][-50:]
print('End of document: "...' + end_text_nrsb + '"')

Respecting sentence boundary
End of document: "...cornerstone of a typical elite European education."

Not respecting sentence boundary
End of document: "...xts used as part of a curriculum, both derive from"


A commonly used strategy to split long documents, especially in the field of Question Answering,
is the sliding window approach. If `split_length=10` and `split_overlap=3`, your documents will look like this:

- doc1 = words[0:10]
- doc2 = words[7:17]
- doc3 = words[14:24]
- ...

You can use this strategy by following the code below.

In [ ]:
# Sliding window approach

preprocessor_sliding_window = PreProcessor(split_overlap=3, split_length=10,
                                           split_respect_sentence_boundary=False)
docs_sliding_window = preprocessor_sliding_window.process(doc_txt)

doc1 = docs_sliding_window[0]['content'][:200]
doc2 = docs_sliding_window[1]['content'][:100]
doc3 = docs_sliding_window[2]['content'][:100]

print('Document 1: "' + doc1 + '..."')
print('Document 2: "' + doc2 + '..."')
print('Document 3: "' + doc3 + '..."')

Document 1: "Classics or classical studies is the study of classical antiquity,..."
Document 2: "of classical antiquity, and in the Western world traditionally refers..."
Document 3: "world traditionally refers to the study of Classical Greek and..."


## Bringing it all together

In [ ]:
all_docs = convert_files_to_dicts(dir_path="data/preprocessing_tutorial")
preprocessor = PreProcessor(
    clean_empty_lines=True,
    clean_whitespace=True,
    clean_header_footer=False,
    split_by="word",
    split_length=100,
    split_respect_sentence_boundary=True,
)
docs = preprocessor.process(all_docs)

print(f"n_files_input: {len(all_docs)}\nn_docs_output: {len(docs)}")

INFO - haystack.utils.preprocessing -  Converting data/preprocessing_tutorial/heavy_metal.docx
INFO - haystack.utils.preprocessing -  Converting data/preprocessing_tutorial/classics.txt
INFO - haystack.utils.preprocessing -  Converting data/preprocessing_tutorial/bert.pdf
100%|██████████| 3/3 [00:00<00:00, 70.32docs/s]

n_files_input: 3
n_docs_output: 163
